In [ ]:
!mkdir data

## Download model and move to data/model

In [ ]:
model_path = "data/model/mobilenet_v2.pth.tar"

## Load MobileNetV2

In [ ]:
import torch
from MobileNetV2 import MobileNetV2
mobile_net = MobileNetV2(n_class=1000)
state_dict = torch.load(model_path, map_location='cpu') # add map_location='cpu' if no gpu
mobile_net.load_state_dict(state_dict)

## Transfer Learning Prerequisites
1. Change output layer
2. Freeze weights

The MobileNetV2 has two very important attributes *features* and *classifier*. 
- *features* defines everything in the neural network until (n-1)th layer. 
- *classifier* defines the output layer

In [ ]:
mobile_net.classifier

Through the output above we can see that the output layer has 1280 inputs and 1000 outputs. In order to apply transfer learning to a new dataset we need to reduce the outputs to the number of the categories in the new dataset.

In [ ]:
import torch.nn as nn
n_class = 5
new_classifier = nn.Sequential(
    nn.Dropout(0.2),
    nn.Linear(mobile_net.last_channel, n_class),
)

## Data download

In [ ]:
import os
from fastai.vision import *
BASE_DATA = Path('data/images/')
categories = ["banana", "apple", "orange"]

In [ ]:
def return_files(category):
    files = !ls {BASE_DATA}/c/*.{jpg,jpeg,png}
    return files

In [ ]:
for c in categories:
    f = c + ".csv"
    download_images(BASE_DATA / f, BASE_DATA/c, max_pics=300)

    # try to open image - if fails, delete
    verify_images(BASE_DATA/c, delete=True, max_size=400)

In [ ]:
data = ImageDataBunch.from_folder(BASE_DATA, 
                                  train=".", 
                                  valid_pct=0.2,
                                  ds_tfms=get_transforms(), 
                                  size=224, 
                                  num_workers=0).normalize(imagenet_stats)

In [ ]:
data.show_batch(rows=4, figsize=(12,12))

## Load Model

In [ ]:
from transfer_model import TransferModel

In [ ]:
n_classes = 3
pretrained_weights = "data/model/mobilenet_v2.pth.tar"
mobile_net = TransferModel(3, pretrained_weights)

## Train model

In [ ]:
learner = Learner(data, mobile_net)

In [ ]:
learner.fit_one_cycle(10)

In [ ]:
interp = ClassificationInterpretation.from_learner(learner)

In [ ]:
interp.plot_confusion_matrix()

# Test images

In [ ]:
img = open_image("apple_test.jpeg")
pred_class,pred_idx,outputs = learner.predict(img)
print(pred_class, pred_idx, outputs)

## Export

In [ ]:
from torch.autograd import Variable
import torch
dummy_input = Variable(torch.randn(1, 3, 224, 224))
# Export the trained model to ONNX
torch.onnx.export(learner.model.cpu(), dummy_input, "fruits.onnx")

In [ ]:
import onnx

model = onnx.load("fruits.onnx")

# Convert Core ML

In [ ]:
from onnx_coreml import convert

"""
IMAGE_NET_MEAN = [0.485, 0.456, 0.406]
IMAGE_NET_STD = [0.229, 0.224, 0.225]

"""

scale = 1.0 / (0.226 * 255.0)
args = dict(
    is_bgr=False,
    red_bias=-(0.485*1/0.229),
    green_bias=-(0.456*1/0.224),
    blue_bias=-(0.406*1/0.225),
    image_scale = scale
)
mlmodel = convert(model, 
                  image_input_names=['0'], 
                  mode="classifier", 
                  image_output_names=['0'], 
                  class_labels=["0", "1", "2"],
                 preprocessing_args=args)

In [ ]:
from coremltools.models import MLModel

spec = mlmodel.get_spec()
new_mlmodel = MLModel(spec)
new_mlmodel

In [ ]:
new_mlmodel.save("fruits_new.mlmodel")